# 1.  Importing the relevant libraries and data 

Let us first start with the usual importing of relevant libraries which will be required for our analysis. We shall import the data into the dataframe name df

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df=pd.read_csv('Placement_Data_Full_Class.csv')
df.head()

,sl_no,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,status,salary
0,1,M,67.00,Others,91.00,Others,Commerce,58.00,Sci&Tech,No,55.0,Mkt&HR,58.80,Placed,270000.0
1,2,M,79.33,Central,78.33,Others,Science,77.48,Sci&Tech,Yes,86.5,Mkt&Fin,66.28,Placed,200000.0
2,3,M,65.00,Central,68.00,Central,Arts,64.00,Comm&Mgmt,No,75.0,Mkt&Fin,57.80,Placed,250000.0
3,4,M,56.00,Central,52.00,Central,Science,52.00,Sci&Tech,No,66.0,Mkt&HR,59.43,Not Placed,NaN
4,5,M,85.80,Central,73.60,Central,Commerce,73.30,Comm&Mgmt,No,96.8,Mkt&Fin,55.50,Placed,425000.0


In [3]:
df_copy=df.copy()

# 2. Data wrangling

Since the raw data could have lots of missing values or other issues, we will be taking care to align the data into simpler more understandable and clear data which will be further helpful to be used in our ML models. Since the columns have not been clearly mentioned as to what they exactly mean, I have made the following assumptions to understand the data better:

* 'ssc_p': 10th Grade percentage
* 'ssc_b':10th Grade board 
* 'hsc_p': 12th Grade percentage 
* 'hsc_b': 12th Grade board 
* 'hsc_s': Higher secondary stream
* 'degree_p': Undergraduate percentage 
* 'degree_t': Undergraduate degree type  
* 'workex': Work experience  
* 'etest_p':Placement test percentage 
* 'specialisation': MBA specialisation 
* 'mba_p': MBA percentage
* 'status': Hiring status 

In [4]:
df['sl_no'].unique().size

215

The data give to us has a total of 215 entries. This is roughly the size of a MBA batch for a particular year of a college. Hence, the data provided could be that for the batch of a particular year.

Let us check the various data types available to us and see if there are any missing values.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215 entries, 0 to 214
Data columns (total 15 columns):
sl_no             215 non-null int64
gender            215 non-null object
ssc_p             215 non-null float64
ssc_b             215 non-null object
hsc_p             215 non-null float64
hsc_b             215 non-null object
hsc_s             215 non-null object
degree_p          215 non-null float64
degree_t          215 non-null object
workex            215 non-null object
etest_p           215 non-null float64
specialisation    215 non-null object
mba_p             215 non-null float64
status            215 non-null object
salary            148 non-null float64
dtypes: float64(6), int64(1), object(8)
memory usage: 25.3+ KB


## Salary

As we can clearly see, the only missing values present are in the salary column. This is because the missing values are corresponding to students to did not get placed in the placement program. 

We will make the assumption that the missing values are filled with 0.

In [6]:
df['salary'].fillna(0,inplace=True)
df['salary'].isna().any()

False

In [25]:
df_copy['salary'].median()

265000.0

So, we have replaced all the missing values with the median value of Rs. 2,65,000

## Gender

Gender column is straightforward with either Male (M) or Female (F) indices.
Since we cannot leave the data in object form, let us replace 

**M with 0** and **F with 1** .

In [8]:
df['gender']=df['gender'].map({'M':0,'F':1})

In [9]:
df.head()

,sl_no,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,status,salary
0,1,0,67.00,Others,91.00,Others,Commerce,58.00,Sci&Tech,No,55.0,Mkt&HR,58.80,Placed,270000.0
1,2,0,79.33,Central,78.33,Others,Science,77.48,Sci&Tech,Yes,86.5,Mkt&Fin,66.28,Placed,200000.0
2,3,0,65.00,Central,68.00,Central,Arts,64.00,Comm&Mgmt,No,75.0,Mkt&Fin,57.80,Placed,250000.0
3,4,0,56.00,Central,52.00,Central,Science,52.00,Sci&Tech,No,66.0,Mkt&HR,59.43,Not Placed,0.0
4,5,0,85.80,Central,73.60,Central,Commerce,73.30,Comm&Mgmt,No,96.8,Mkt&Fin,55.50,Placed,425000.0


## ssc_b and hsc_b

Here, we are presented with the data of the 10th board. Let us check the unique entries and their counts we are presented with.

In [10]:
df['ssc_b'].value_counts()

Central    116
Others      99
Name: ssc_b, dtype: int64

As we can observe, the unique entries are either Central or others. By Central, the indication must be that the board is CBSE. While others could mean variety of boards such as state board, international board or ICSC board. The candidates with central board is slightly higher as compared to other boards. 

We shall perform the exact same treatment as we did for gender.

**Central : 1
Others : 0**

In [11]:
df['ssc_b']=df['ssc_b'].map({'Central':1,'Others':0})

In [12]:
df['hsc_b']=df['hsc_b'].map({'Central':1,'Others':0})

## hsc_s

This column deals with the specialisation subjects taken by students when they enter 11th grade. Let us see the various entries alongwith their counts.

In [13]:
df['hsc_s'].value_counts()

Commerce    113
Science      91
Arts         11
Name: hsc_s, dtype: int64

As can be seen, students with commerce specialisation was highest followed by science and arts.

We shall use one-hot-encoding for this particular feature. This can be done using the pd.get_dummies function.

In [14]:
df_subjects=pd.get_dummies(df['hsc_s'])
df=df.merge(df_subjects,on=df.index)

In [15]:
df.drop('key_0',axis=1,inplace=True)
df.drop('hsc_s',axis=1,inplace=True)

In [16]:
df.columns

Index(['sl_no', 'gender', 'ssc_p', 'ssc_b', 'hsc_p', 'hsc_b', 'degree_p',
       'degree_t', 'workex', 'etest_p', 'specialisation', 'mba_p', 'status',
       'salary', 'Arts', 'Commerce', 'Science'],
      dtype='object')

In [17]:
df=df[['sl_no', 'gender', 'ssc_p', 'ssc_b', 'hsc_p', 'hsc_b', 'Arts', 'Commerce','Science','degree_p',
       'degree_t', 'workex', 'etest_p', 'specialisation', 'mba_p', 'status',
       'salary']]

## degree_t

Let us check the various types of Undergrad degrees available in the data.

In [18]:
df['degree_t'].value_counts()

Comm&Mgmt    145
Sci&Tech      59
Others        11
Name: degree_t, dtype: int64

As we can see, most students had their undergraduate degrees in commerce and management followed by science and then others. Just as we did for higher secondary subjects, we shall one hot encode the results of undergrad degrees aswell.

In [19]:
df_deg=pd.get_dummies(df['degree_t'])
df=df.merge(df_deg,on=df.index)

In [20]:
df.drop('key_0',axis=1,inplace=True)
df.drop('degree_t',axis=1,inplace=True)

In [21]:
df.columns

Index(['sl_no', 'gender', 'ssc_p', 'ssc_b', 'hsc_p', 'hsc_b', 'Arts',
       'Commerce', 'Science', 'degree_p', 'workex', 'etest_p',
       'specialisation', 'mba_p', 'status', 'salary', 'Comm&Mgmt', 'Others',
       'Sci&Tech'],
      dtype='object')

In [22]:
df=df[['sl_no', 'gender', 'ssc_p', 'ssc_b', 'hsc_p', 'hsc_b', 'Arts',
       'Commerce', 'Science', 'degree_p','Comm&Mgmt', 'Others',
       'Sci&Tech','workex', 'etest_p',
       'specialisation', 'mba_p', 'status', 'salary']]

In [23]:
df

,sl_no,gender,ssc_p,ssc_b,hsc_p,hsc_b,Arts,Commerce,Science,degree_p,Comm&Mgmt,Others,Sci&Tech,workex,etest_p,specialisation,mba_p,status,salary
0,1,0,67.00,0,91.00,0,0,1,0,58.00,0,0,1,No,55.00,Mkt&HR,58.80,Placed,270000.0
1,2,0,79.33,1,78.33,0,0,0,1,77.48,0,0,1,Yes,86.50,Mkt&Fin,66.28,Placed,200000.0
2,3,0,65.00,1,68.00,1,1,0,0,64.00,1,0,0,No,75.00,Mkt&Fin,57.80,Placed,250000.0
3,4,0,56.00,1,52.00,1,0,0,1,52.00,0,0,1,No,66.00,Mkt&HR,59.43,Not Placed,0.0
4,5,0,85.80,1,73.60,1,0,1,0,73.30,1,0,0,No,96.80,Mkt&Fin,55.50,Placed,425000.0
5,6,0,55.00,0,49.80,0,0,0,1,67.25,0,0,1,Yes,55.00,Mkt&Fin,51.58,Not Placed,0.0
6,7,1,46.00,0,49.20,0,0,1,0,79.00,1,0,0,No,74.28,Mkt&Fin,53.29,Not Placed,0.0
7,8,0,82.00,1,64.00,1,0,0,1,66.00,0,0,1,Yes,67.00,Mkt&Fin,62.14,Placed,252000.0
8,9,0,73.00,1,79.00,1,0,1,0,72.00,1,0,0,No,91.34,Mkt&Fin,61.29,Placed,231000.0
9,10,0,58.00,1,70.00,1,0,1,0,61.00,1,0,0,No,54.00,Mkt&Fin,52.21,Not Placed,0.0


## workex

This column tells us if a student has any work experience prior to joining the MBA programme.

We shall map the entries as

**Yes:1
No:0**

In [24]:
df['workex']=df['workex'].map({'Yes':1,'No':0})
df.head()

,sl_no,gender,ssc_p,ssc_b,hsc_p,hsc_b,Arts,Commerce,Science,degree_p,Comm&Mgmt,Others,Sci&Tech,workex,etest_p,specialisation,mba_p,status,salary
0,1,0,67.00,0,91.00,0,0,1,0,58.00,0,0,1,0,55.0,Mkt&HR,58.80,Placed,270000.0
1,2,0,79.33,1,78.33,0,0,0,1,77.48,0,0,1,1,86.5,Mkt&Fin,66.28,Placed,200000.0
2,3,0,65.00,1,68.00,1,1,0,0,64.00,1,0,0,0,75.0,Mkt&Fin,57.80,Placed,250000.0
3,4,0,56.00,1,52.00,1,0,0,1,52.00,0,0,1,0,66.0,Mkt&HR,59.43,Not Placed,0.0
4,5,0,85.80,1,73.60,1,0,1,0,73.30,1,0,0,0,96.8,Mkt&Fin,55.50,Placed,425000.0


## specialisation

This column tells us about the type of MBA program the student is currently enrolled in. The value counts for each specialisation are as follows.

In [27]:
df['specialisation'].value_counts()

Mkt&Fin    120
Mkt&HR      95
Name: specialisation, dtype: int64

Hence, majority of the students were in Marketing and Finance while the remaining in Marketing and HR.

We should encode the data as 

**Mkt&Fin : 1
Mkt&HR : 0**

In [29]:
df['specialisation']=df['specialisation'].map({'Mkt&Fin':1,'Mkt&HR':0})

## status

This column tells us whether the student got placed in the placement program. This is our target variable which is important for training and testing purpose in the ML phase.

Let us map the terms placed and unplaced as 

**Placed : 1
Not Placed : 0**

In [31]:
df['status'].value_counts()

Placed        148
Not Placed     67
Name: status, dtype: int64

In [32]:
df['status']=df['status'].map({'Placed':1,'Not Placed':0})

Now, we can consider the data to be cleaned completely off any issues and can be safely used for visualisation and machine learning purposes. Let us take a peek at how the final dataframe looks like.

In [33]:
df.head()

,sl_no,gender,ssc_p,ssc_b,hsc_p,hsc_b,Arts,Commerce,Science,degree_p,Comm&Mgmt,Others,Sci&Tech,workex,etest_p,specialisation,mba_p,status,salary
0,1,0,67.00,0,91.00,0,0,1,0,58.00,0,0,1,0,55.0,0,58.80,1,270000.0
1,2,0,79.33,1,78.33,0,0,0,1,77.48,0,0,1,1,86.5,1,66.28,1,200000.0
2,3,0,65.00,1,68.00,1,1,0,0,64.00,1,0,0,0,75.0,1,57.80,1,250000.0
3,4,0,56.00,1,52.00,1,0,0,1,52.00,0,0,1,0,66.0,0,59.43,0,0.0
4,5,0,85.80,1,73.60,1,0,1,0,73.30,1,0,0,0,96.8,1,55.50,1,425000.0
